# 使用重复元素的网络（VGG）

AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然AlexNet指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。

本节介绍VGG，它的名字来源于论文作者所在的实验室Visual Geometry Group [1]。VGG提出了可以通过重复使用简单的基础块来构建深度模型的思路。

## VGG块

VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。使用`vgg_block`函数直接生成所有的vgg块。


*注：原文使用 `vgg_block` 函数来实现这个基础的VGG块，它可以指定卷积层的数量 `num_convs` 和输出通道数 `num_channels` 。但是因为 PyTorch 与 MxNet 在实现模型上存在差异，这里采用 torchvision 中的构建方式。*

In [0]:
import d2ltorch as d2ltorch
import torch
from torch import nn, optim
from torchsummary import summary

def vgg_block(in_channels, cfg):
    blk = []
    for v in cfg:
        if v == 'M':
            blk += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            blk += [nn.Conv2d(in_channels, v, kernel_size=3, padding=1), nn.ReLU(inplace=True)]
            in_channels = v

    return nn.Sequential(*blk)

## VGG网络

与AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个`vgg_block`，其超参数由变量`conv_arch`定义。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则跟AlexNet中的一样。

现在我们构造一个VGG网络。它有5个卷积块，前2块使用单卷积层，而后3块使用双卷积层。第一块的输出通道是64，之后每次对输出通道数翻倍，直到变为512。因为这个网络使用了8个卷积层和3个全连接层，所以经常被称为VGG-11。

In [0]:
cfg = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

下面我们实现VGG-11。

In [0]:
class MyVGG(nn.Module):
    def __init__(self, in_channels, out_channels, cfg, num_classes=10, **kwargs):
        super(MyVGG, self).__init__(**kwargs)
        
        # 卷积层部分
        self.conv = vgg_block(in_channels, cfg)
        
        # torchvision中vgg的实现在进入全连接层之前有一层nn.AdaptiveAvgPool2d((7, 7))保证输出大小为7*7
        # 这里默认输出大小为7*7
        
        # 全连接层部分
        self.linear = nn.Sequential(
            nn.Linear(out_channels*7*7, 4096), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(4096, 4096), nn.ReLU(inplace=True), nn.Dropout(0.5),
            nn.Linear(4096, 10)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x)
        return x

下面构造一个高和宽均为224的单通道数据样本来观察每一层的输出形状。

In [21]:
net = MyVGG(in_channels=1, out_channels=512, cfg=cfg)
summary(net, (1, 224, 224), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             640
              ReLU-2         [-1, 64, 224, 224]               0
         MaxPool2d-3         [-1, 64, 112, 112]               0
            Conv2d-4        [-1, 128, 112, 112]          73,856
              ReLU-5        [-1, 128, 112, 112]               0
         MaxPool2d-6          [-1, 128, 56, 56]               0
            Conv2d-7          [-1, 256, 56, 56]         295,168
              ReLU-8          [-1, 256, 56, 56]               0
            Conv2d-9          [-1, 256, 56, 56]         590,080
             ReLU-10          [-1, 256, 56, 56]               0
        MaxPool2d-11          [-1, 256, 28, 28]               0
           Conv2d-12          [-1, 512, 28, 28]       1,180,160
             ReLU-13          [-1, 512, 28, 28]               0
           Conv2d-14          [-1, 512,

注：  
第一个vgg块: Conv2d-1 &nbsp;&nbsp;→ MaxPool2d-3 &nbsp;，输出形状：[-1, 64, 112, 112]  
第二个vgg块: Conv2d-4 &nbsp;&nbsp;→ MaxPool2d-6 &nbsp;，输出形状：[-1, 128, 56, 56]  
第三个vgg块: Conv2d-7 &nbsp;&nbsp;→ MaxPool2d-11，输出形状：[-1, 256, 28, 28]  
第四个vgg块: Conv2d-12 → MaxPool2d-16，输出形状：[-1, 512, 14, 14]  
第五个vgg块: Conv2d-17 → MaxPool2d-21，输出形状：[-1, 512, 7, 7]  

可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

## 获取数据和训练模型

因为VGG-11计算上比AlexNet更加复杂，出于测试的目的我们构造一个通道数更小，或者说更窄的网络在Fashion-MNIST数据集上进行训练。

In [0]:
small_cfg = [16, 'M', 32, 'M', 64, 64, 'M', 64, 64, 'M', 64, 64, 'M']
net = MyVGG(in_channels=1, out_channels=64, cfg=small_cfg)

除了使用了稍大些的学习率，模型训练过程与上一节的AlexNet中的类似。

In [23]:
root = '~/dataset/'
lr, num_epochs, batch_size, device = 0.05, 5, 128, d2lt.try_gpu()
if device == 'gpu':
    net.cuda()

d2lt.params_init(net, init=nn.init.xavier_uniform_)
optimizer = optim.SGD(net.parameters(), lr=lr)
train_iter, test_iter = d2lt.load_data_fashion_mnist(root, batch_size, resize=224)
d2lt.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on gpu
epoch 1, loss 0.0071, train acc 0.659, test acc 0.834, time 95.5 sec
epoch 2, loss 0.0031, train acc 0.855, test acc 0.844, time 100.1 sec
epoch 3, loss 0.0025, train acc 0.881, test acc 0.881, time 100.1 sec
epoch 4, loss 0.0023, train acc 0.894, test acc 0.884, time 99.9 sec
epoch 5, loss 0.0020, train acc 0.904, test acc 0.884, time 99.8 sec


## 小结

* VGG-11通过5个可以重复使用的卷积块来构造网络。根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。

## 练习

* 与AlexNet相比，VGG通常计算慢很多，也需要更多的内存或显存。试分析原因。
* 尝试将Fashion-MNIST中图像的高和宽由224改为96。这在实验中有哪些影响？
* 参考VGG论文里的表1来构造VGG其他常用模型，如VGG-16和VGG-19 [1]。



## 参考文献

[1] Simonyan, K., & Zisserman, A. (2014). Very deep convolutional networks for large-scale image recognition. arXiv preprint arXiv:1409.1556.

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/1277)

![](../img/qr_vgg.svg)